In [1]:
import numpy as np
import os, sys, glob, abc

def read(filelist):
    data = []
    for f in sorted(filelist):
        x = np.load(f)
        if len(data) == 0: data = x.copy()
        else: data = np.concatenate([data, x])
    return data

# Where is the dataset stored?
dataset = "/data/i3store/users/mjlarson/student_data"

# Read in all of the data files
data_files = dataset + "/IC86_*exp.npy"
data = read(glob.glob(data_files))

# Read in all of the MC files 
sim_files = dataset + "/IC86*MC*npy"
sim = read(glob.glob(sim_files))

# Set the angular error floor to 0.2 degrees
data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)

data['ra']=np.random.uniform(0,2*np.pi,size=len(data))

In [2]:
import mla.core
from mla.spectral import *
import threeML
#sp= PowerLaw(1,1e-15,2)
sp=threeML.Powerlaw()


/data/disk01/home/jasonfan/threeML/threeML/__init__.py:12: UserWarning: No DISPLAY variable set. Using backend for graphics without display (Agg)
  warnings.warn("No DISPLAY variable set. Using backend for graphics without display (Agg)")
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/astromodels/core/parameter.py:555: UserWarning: We have set the min_value of K to 1e-99 because there was a postive transform
  warnings.warn('We have set the min_value of %s to 1e-99 because there was a postive transform' % self.path)
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/astromodels/core/parameter.py:555: UserWarning: We have set the min_value of xc to 1e-99 because there was a postive transform
  warnings.warn('We have set the min_value of %s to 1e-99 because there was a postive transform' % self.path)

Bad key "text.kerning_factor" on line 4 in
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/matplotlib/mpl-data/st

In [18]:
from mla.spectral import *
from mla import injection
sp= PowerLaw(1,1e-15,2)
dec = 22.0145*np.pi/180
ra = 83.63*np.pi/180
inj = injection.PSinjector(sp,sim)
inj.set_source_location(ra,dec)
sample=inj.sample_nevents(5)

In [25]:
from imp import reload
import scipy.optimize
reload(mla.core)
sp.K=1e-8
sp.index=-2
background = np.random.choice(data, 50).copy()

test=mla.core.LLH_point_source(np.radians(83.63) ,np.radians(22.0145) , data , sim , 300 , sp)
test.update_data(background,1000/3600/24)
test.add_injection(sample)
import warnings
warnings.filterwarnings("ignore")

In [26]:
def fit(ns):
    return -test.eval_llh_ns(ns)[1]
result=scipy.optimize.minimize(fit,5,bounds=[(0,test.N)])

In [27]:
result

      fun: -86.49150679948045
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([2.84217094e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 10
      nit: 4
   status: 0
  success: True
        x: array([3.9999345])

In [29]:
background

array([(120677, 58070736, 0, 5.25732096,  0.03593391, 3.27887048, 1.60692753, 56185.50334182, 3.13726936, 0.00763364),
       (121058, 45506694, 0, 5.86299103,  0.53818747, 1.3661355 , 2.10786993, 56255.37831265, 2.70884331, 0.02011854),
       (120369, 21962461, 0, 4.28346479,  0.07484773, 5.09608293, 1.64682287, 56099.73365959, 3.08825133, 0.01151901),
       (120425,  4614079, 0, 0.59821528, -0.77797966, 1.10699267, 0.79289006, 56116.88633981, 5.15752004, 0.00349066),
       (120799, 35169149, 0, 5.87725629,  0.48983277, 0.17451905, 2.05951274, 56212.16334014, 2.9925499 , 0.0062995 ),
       (122140,  9206866, 0, 5.46593821,  0.23255183, 3.94383419, 1.80334079, 56386.08516715, 3.23014191, 0.01124464),
       (120760, 66424257, 0, 4.37836827, -0.89873491, 1.71287888, 0.67288577, 56203.8632761 , 5.38361771, 0.00349066),
       (121436, 39331278, 0, 5.98549889, -0.1548552 , 3.83077089, 1.41678039, 56284.22888139, 4.00950706, 0.004215  ),
       (120522, 24472695, 0, 6.227723  , -0.6231